# CSE 6040, Fall 2015 [28]: K-means Clustering, Part 2

Last time, we implemented the basic version of K-means. In this lecture we will explore some advanced techniques
to improve performance of K-means.

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns

%matplotlib inline

### Read in data

In [ ]:
df = pd.read_csv ('http://vuduc.org/cse6040/logreg_points_train.csv')
points = df.as_matrix (['x_1', 'x_2'])
labels = df['label'].as_matrix ()
n = points.shape[0]
d = points.shape[1]
k = 2

In [ ]:
def init_centers(X, k):
    sampling = np.random.randint(0, n, k)
    return X[sampling, :]

### Fast implementation of the distance matrix computation
The idea is that $$||(x - c)||^2 = ||x||^2 -  2\langle x, c \rangle + ||c||^2 $$
This has many advantages.
1. The centers are fixed (during a single iteration), so only needs to compute once
2. Data points are usually sparse, but centers are not
3. If implement cleverly, we don't need to use for loops

In [ ]:
def compute_d2(X, centers):
    D = np.empty((n, k))   
    for i in range(n):
        D[i, :] = np.linalg.norm(X[i,:] - centers, axis=1) ** 2
    
    return D

In [ ]:
def compute_d2_fast(X, centers):

    # @YOUSE: compute a length-n array, where each entry is the square of norm of a point
    first_term = 

    # @YOUSE: compute a (n * k) matrix, where entry (i,j) is the two times of inner product of row i of X and row j of centers
    second_term = 

    # @YOUSE: compute a length-k array, where each entry is the square of norm of a center
    third_term = 
    
    D = np.tile(first_term, (k, 1)).T - second_term + np.tile(third_term, (n,1))
    D[D < 0] = 0
    
    return D

In [ ]:
centers = init_centers(points, k)
%timeit D = compute_d2(points, centers)
%timeit D = compute_d2_fast(points, centers)

In [ ]:
def cluster_points(D): 
    return np.argmin(D, axis=1)

In [ ]:
def update_centers(X, clustering):
    centers = np.empty((k, d))
    for i in range(k):
        members = (clustering == i)
        if any(members):
            centers[i, :] = np.mean(X[members, :], axis=0)
    return centers

In [ ]:
def WCSS(D):
    min_val = np.amin(D, axis=1)
    return np.sum(min_val)

In [ ]:
def has_converged(old_centers, centers):
    return set([tuple(x) for x in old_centers]) == set([tuple(x) for x in centers])

In [ ]:
def kmeans(X, k):
    old_centers = init_centers(X, k)
    centers = init_centers(X, k)
    i = 1
    while not has_converged(old_centers, centers):
        old_centers = centers
        D = compute_d2_fast(X, centers)
        clustering = cluster_points(D)
        centers = update_centers(X, clustering)
        print "iteration", i, "WCSS = ", WCSS(D)
        i += 1
    return clustering

In [ ]:
clustering = kmeans(points, k)

In [ ]:
df['clustering'] = clustering
sns.lmplot(data=df, x="x_1", y="x_2", hue="clustering", fit_reg=False,)

### K-means implementation in Scipy

In [ ]:
from scipy.cluster.vq import kmeans,vq

In [ ]:
centers, distortion = kmeans(points, k)
clustering,_ = vq(points, centers)

In [ ]:
df['clustering'] = clustering
sns.lmplot(data=df, x="x_1", y="x_2", hue="clustering", fit_reg=False,)

### Elbow method to determine a good k

In [ ]:
df_kcurve = pd.DataFrame(columns = ['k', 'distortion']) 
for k in range(1,10):
    _, distortion = kmeans(points, k)
    df_kcurve.loc[k] = [k, distortion]

In [ ]:
df_kcurve.plot(x="k", y="distortion")

### Exercise: implement K-means++
You only need to modify the init_centers function.

In [ ]:
def init_centers_kplusplus(X, k):
    # @YOUSE: implement the initialization step in k-means++ 
    pass